In [ ]:
# !cd ..
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls

In [ ]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import cv2
import random
import time
from tqdm import tqdm
# (OPTIONAL) step
# Preparing the Dataset
# Convert all images to .jpg (This is not necessary but made my life easier down the line)

from PIL import Image
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import greatbarrierreef

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("experiment",)
# cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")# initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.0005 
# cfg.SOLVER.MAX_ITER = 5000   # 10000 iterations seems good enough, but you can certainly train longer
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (Person, Helmet, Car)
cfg.MODEL.WEIGHTS = "../input/detectron2-fasterrcnn-model-5000-epc/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("experiment",)
predictor = DefaultPredictor(cfg)

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path (this can be local or on colossus)

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def detect(image_np):
    """Detect COTS from a given numpy image."""

#     input_tensor = np.expand_dims(image_np, 0)
#     start_time = time.time()
    detections = predictor(image_np)
    return detections

In [ ]:
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
DETECTION_THRESHOLD = 0.3

submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:

    height, width, _ = image_np.shape
    detections = detect(image_np)

    num_detections = len(detections["instances"].pred_boxes)
    predictions = []
    for index in range(num_detections):
        score = detections["instances"].scores.to("cpu").numpy()[index]
        if score < DETECTION_THRESHOLD:
            continue

        x1, y1, x2, y2 = np.asarray(detections["instances"].pred_boxes)[0].to("cpu").numpy()

        predictions.append('{:.2f} {} {} {} {}'.format(score, int(x1), int(y1), int(x2), int(y2)))
    
    
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)

In [ ]:
# from pandas.compat import StringIO

In [ ]:
# import pandas
# print(pandas.__version__)

In [ ]:
# !cat /opt/conda/lib/python3.7/site-packages/pandas/io/formats/csvs.py

In [ ]:
# !cat /opt/conda/lib/python3.7/site-packages/pandas/compat/__init__.py

In [ ]:
#     print(detections)
#     print("======================================================")
#     preds = detections["instances"].to("cpu")
#     print(f"preds cpu: {preds}")
#     print("======================================================")
# #     print(f"num of instances: {len(detections["instances"].pred_boxes)}")
#     print(len(detections["instances"].pred_boxes))
#     print("======================================================")
#     print(np.asarray(detections["instances"].pred_boxes)[0].to("cpu").numpy())
# #     print("======================================================")
# #     print(detections["instances"].pred_boxes.to("cpu").numpy())
#     print("======================================================")
#     print(detections["instances"].scores.to("cpu").numpy())
#     print("======================================================")
